## Batch processing with Argo Worfklows

In this notebook we will dive into how you can run batch processing with Argo Workflows and Seldon Core.

Dependencies:

* Seldon core installed as per the docs with an ingress
* Argo Workfklows installed in cluster (and argo CLI for commands)


## Argo Workflows Example

Let's try an argo workflows example to see intuitively how it works. 

In this case we will trigger a workflow with 3 steps (first one will execute and the other two jobs are dependent on that)

In [1]:
mkdir -p assets

In [12]:
%%writefile assets/argo-example.yaml
---
apiVersion: argoproj.io/v1alpha1
kind: Workflow
metadata:
  generateName: steps-
spec:
  entrypoint: hello-hello-hello
  # This spec contains two templates: hello-hello-hello and whalesay
  templates:
  - name: hello-hello-hello
    # Instead of just running a container
    # This template has a sequence of steps
    steps:
    - - name: hello1            # hello1 is run before the following steps
        template: whalesay
        arguments:
          parameters:
          - name: message
            value: "hello1"
    - - name: hello2a           # double dash => run after previous step
        template: whalesay
        arguments:
          parameters:
          - name: message
            value: "hello2a"
      - name: hello2b           # single dash => run in parallel with previous step
        template: whalesay
        arguments:
          parameters:
          - name: message
            value: "hello2b"
  # This is the same template as from the previous example
  - name: whalesay
    inputs:
      parameters:
      - name: message
    container:
      image: docker/whalesay
      command: [cowsay]
      args: ["{{inputs.parameters.message}}"]
        

Overwriting assets/argo-example.yaml


In [13]:
!argo submit assets/argo-example.yaml

Name:                steps-9tgj9
Namespace:           default
ServiceAccount:      default
Status:              Pending
Created:             Fri Apr 17 08:27:14 +0100 (8 hours ago)


In [14]:
!argo list

NAME          STATUS      AGE   DURATION   PRIORITY
steps-9tgj9   Succeeded   8h    3m         0


In [20]:
output=!argo list | grep steps
WF_NAME=output[0].split()[0]
print(WF_NAME)

steps-9tgj9


In [23]:
!argo get $WF_NAME

Name:                steps-9tgj9
Namespace:           default
ServiceAccount:      default
Status:              Succeeded
Created:             Fri Apr 17 08:27:14 +0100 (8 hours ago)
Started:             Fri Apr 17 08:27:14 +0100 (8 hours ago)
Finished:            Fri Apr 17 08:30:48 +0100 (8 hours ago)
Duration:            3 minutes 34 seconds

STEP                                PODNAME                 DURATION  MESSAGE
 ✔ steps-9tgj9 (hello-hello-hello)                                    
 ├---✔ hello1 (whalesay)            steps-9tgj9-3240403473  3m        
 └-·-✔ hello2a (whalesay)           steps-9tgj9-3510808138  3s        
   └-✔ hello2b (whalesay)           steps-9tgj9-3494030519  5s        


In [22]:
!argo logs -w $WF_NAME

hello1:	 ________ 
hello1:	< hello1 >
hello1:	 -------- 
hello1:	    \
hello1:	     \
hello1:	      \     
hello1:	                    ##        .            
hello1:	              ## ## ##       ==            
hello1:	           ## ## ## ##      ===            
hello1:	       /""""""""""""""""___/ ===        
hello1:	  ~~~ {~~ ~~~~ ~~~ ~~~~ ~~ ~ /  ===- ~~~   
hello1:	       \______ o          __/            
hello1:	        \    \        __/             
hello1:	          \____\______/   
hello2a:	 _________ 
hello2a:	< hello2a >
hello2a:	 --------- 
hello2a:	    \
hello2a:	     \
hello2a:	      \     
hello2a:	                    ##        .            
hello2a:	              ## ## ##       ==            
hello2a:	           ## ## ## ##      ===            
hello2a:	       /""""""""""""""""___/ ===        
hello2a:	  ~~~ {~~ ~~~~ ~~~ ~~~~ ~~ ~ /  ===- ~~~   
hello2a:	       \______ o          __/            
hello2a:	        \    \        __/             
hello2a:	          \____\__

In [25]:
!argo delete $WF_NAME

Workflow 'steps-9tgj9' deleted


## Seldon Core Batch 
Now we can leverage this functionality by using seldon core batch

In [482]:
%%writefile assets/seldon-batch.yaml
---
apiVersion: argoproj.io/v1alpha1
kind: Workflow
metadata:
  generateName: seldon-batch-
spec:
  entrypoint: seldon-batch-process
  templates:
  - name: seldon-batch-process
    steps:
    - - name: create-seldon-resource            
        template: create-seldon-resource-template
    - - name: wait-seldon-resource
        template: wait-seldon-resource-template
    - - name: process-batch-inputs
        template: process-batch-inputs-template
  - name: create-seldon-resource-template
    resource:
      action: create
      manifest: |
        apiVersion: machinelearning.seldon.io/v1
        kind: SeldonDeployment
        metadata:
          name: "sklearn-{{workflow.uid}}"
          ownerReferences:
          - apiVersion: argoproj.io/v1alpha1
            blockOwnerDeletion: true
            kind: Workflow
            name: "{{workflow.name}}"
            uid: "{{workflow.uid}}"
        spec:
          name: "sklearn-{{workflow.uid}}"
          predictors:
            - graph:
                children: []
                implementation: SKLEARN_SERVER
                modelUri: gs://seldon-models/sklearn/iris
                name: classifier
              name: default
              replicas: 1
  - name: wait-seldon-resource-template
    script:
      image: seldonio/core-builder:0.14
      command: [bash]
      source: |
        kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=sklearn -o jsonpath='{.items[0].metadata.name}')
  - name: process-batch-inputs-template
    script:
      image: seldonio/seldon-core-s2i-python3:1.1.1-SNAPSHOT
      command: [python]
      source: |
        from seldon_core.seldon_client import SeldonClient
        import numpy as np
        import time
        time.sleep(10)
        sc = SeldonClient(
            gateway_endpoint="istio-ingressgateway.istio-system.svc.cluster.local",
            deployment_name="sklearn-{{workflow.name}}",
            namespace="default")
        for i in range(10):
            data = np.array([[i, i, i, i]])
            output = sc.predict(data=data)
            print(output.response)
            

Overwriting assets/seldon-batch.yaml


In [483]:
!argo submit assets/seldon-batch.yaml

Name:                seldon-batch-5jstp
Namespace:           default
ServiceAccount:      default
Status:              Pending
Created:             Fri Apr 17 18:56:48 +0100 (now)


In [484]:
!argo list

NAME                 STATUS    AGE   DURATION   PRIORITY
seldon-batch-5jstp   Running   1s    1s         0


In [485]:
output=!argo list | grep seldon-batch
WF_NAME=output[0].split()[0]
print(WF_NAME)

seldon-batch-5jstp


In [496]:
!argo get $WF_NAME

Name:                seldon-batch-5jstp
Namespace:           default
ServiceAccount:      default
Status:              Failed
Message:             child 'seldon-batch-5jstp-884301607' failed
Created:             Fri Apr 17 18:56:48 +0100 (9 seconds ago)
Started:             Fri Apr 17 18:56:48 +0100 (9 seconds ago)
Finished:            Fri Apr 17 18:56:55 +0100 (2 seconds ago)
Duration:            7 seconds

STEP                                                             PODNAME                        DURATION  MESSAGE
 ✖ seldon-batch-5jstp (seldon-batch-process)                                                              child 'seldon-batch-5jstp-884301607' failed
 ├---✔ create-seldon-resource (create-seldon-resource-template)  seldon-batch-5jstp-4096744685  2s        
 └---✖ wait-seldon-resource (wait-seldon-resource-template)      seldon-batch-5jstp-884301607   3s        failed with exit code 1


In [497]:
!argo logs -w $WF_NAME

create-seldon-resource:	time="2020-04-17T17:56:49Z" level=info msg="Starting Workflow Executor" version=vv2.7.4+50b209c.dirty
create-seldon-resource:	time="2020-04-17T17:56:49Z" level=info msg="Creating a docker executor"
create-seldon-resource:	time="2020-04-17T17:56:49Z" level=info msg="Executor (version: vv2.7.4+50b209c.dirty, build_date: 2020-04-16T16:37:57Z) initialized (pod: default/seldon-batch-5jstp-4096744685) with template:\n{\"name\":\"create-seldon-resource-template\",\"arguments\":{},\"inputs\":{},\"outputs\":{},\"metadata\":{},\"resource\":{\"action\":\"create\",\"manifest\":\"apiVersion: machinelearning.seldon.io/v1\\nkind: SeldonDeployment\\nmetadata:\\n  name: \\\"sklearn-9b5a2d3b-619b-4ddb-b52d-3398b7d447be\\\"\\n  ownerReferences:\\n  - apiVersion: argoproj.io/v1alpha1\\n    blockOwnerDeletion: true\\n    kind: Workflow\\n    name: \\\"seldon-batch-5jstp\\\"\\n    uid: \\\"9b5a2d3b-619b-4ddb-b52d-3398b7d447be\\\"\\nspec:\\n  name: \\\"sklearn-9b5a2d3b-619b-4ddb-b52d-

In [468]:
outputs = !(argo logs -w $WF_NAME --no-color | grep "process-batch-inputs" | cut -c 23-)
for o in outputs:
    print(o)

In [498]:
!argo delete $WF_NAME

Workflow 'seldon-batch-5jstp' deleted
